# Ahsanul Choudhury

## DATA 602 HW 12

This homework will get your feet wet with some basic parallel computing approaches.  

**Do both of the following:***

1.   Take your solution from Homework 11 and complete the Monte Carlo step (step 6) in parallel.  There are many ways you can go about doing this, and I'm not looking for anything too complicated.  If you can get multiple processes crunching the data together, that is great.  Using IPython’s built-in tools would be a great method

2.   Compare the timing for your solution in homework 11 and this parallel solution.  This is similar to what you did in homeworks 6 and 7.  Ideally, you'll see some speed improvement.  The amount you see will largely be based the capabilities of your hardware, and less on the software implementation.  There is additional overhead for running an operation in parallel, so speed gains will be more obvious with a larger number of calculations.

As with the last homework, you will submit this to me as an IPython notebook.  Include the results of your comparison there, along with everything else (code, charts, graphs, etc.)

**HW 11 Approach**

In [1]:
import timeit

In [2]:
s = """\
import pandas as pd
import numpy as np

apple = pd.read_csv('https://raw.githubusercontent.com/choudhury1023/DATA-602/master/apple.2011.csv', skiprows = 1,
                    names = ['date', 'price', 'pct_change'])


## Calculate the mean and standard deviation.

apple['pct_change'] = pd.to_numeric(apple['pct_change'], errors='coerce')
mu = np.mean(apple['pct_change'])
sigma = np.std(apple['pct_change'])


##Starting price:

last_price = apple['price'].iloc[-1]


##Random number generator and 20 random numbers:

np.random.seed(7)
rand_gen = np.random.normal(loc= mu, scale= sigma, size=20)


##Hypothetical price after 20 days:

def get_price(sim_price, days, mu, sigma):
    return sim_price * np.prod(np.random.normal(loc = mu, scale = sigma, size = days) + 1)

np.random.seed(7)
get_price(last_price, 20, mu ,sigma)


##Simulate 10,000 times

results = []
for i in range(10000):
    results.append(get_price(last_price, 20, mu ,sigma))
        
"""

t = timeit.Timer(stmt=s)
t = t.timeit(1)
print t, 'seconds'


1.42377347633 seconds


**Parallel Computing Approach**

In [3]:
import ipyparallel
clients = ipyparallel.Client()
clients.block = True
print clients.ids
dview = clients.direct_view()

[0, 1, 2, 3]


In [4]:
%%px

import timeit

st = """\

import pandas as pd
import numpy as np

apple = pd.read_csv('https://raw.githubusercontent.com/choudhury1023/DATA-602/master/apple.2011.csv', skiprows = 1,
                    names = ['date', 'price', 'pct_change'])


## Calculate the mean and standard deviation.

apple['pct_change'] = pd.to_numeric(apple['pct_change'], errors='coerce')
mu = np.mean(apple['pct_change'])
sigma = np.std(apple['pct_change'])



##Starting price:

last_price = apple['price'].iloc[-1]


##Random number generator and 20 random numbers:

np.random.seed(7)
rand_gen = np.random.normal(loc= mu, scale= sigma, size=20)


##Hypothetical price after 20 days:

def get_price(sim_price, days, mu, sigma):
    return sim_price * np.prod(np.random.normal(loc = mu, scale = sigma, size = days) + 1)

np.random.seed(7)
get_price(last_price, 20, mu ,sigma)


##Simulate 10,000 times

results = []
for i in range(2500):
    results.append(get_price(last_price, 20, mu ,sigma))
        
"""
t1 = timeit.Timer(stmt=st)
t1 = t1.timeit(1)
print t1, 'seconds'

[stdout:0] 0.131585906905 seconds
[stdout:1] 0.114187102557 seconds
[stdout:2] 0.105262378427 seconds
[stdout:3] 0.104261300125 seconds


In [5]:
t1 = dview.gather('t1')
print 'HW11 time', t, 'seconds,', 'parallel computing time', sum (t1), 'seconds'

HW11 time 1.42377347633 seconds, parallel computing time 0.455296688015 seconds
